## TC 5033
### Word Embeddings

<br>

#### Activity 3b: Text Classification using RNNs and AG_NEWS dataset in PyTorch
<br>

- Objective:
    - Understand the basics of Recurrent Neural Networks (RNNs) and their application in text classification.
    - Learn how to handle a real-world text dataset, AG_NEWS, in PyTorch.
    - Gain hands-on experience in defining, training, and evaluating a text classification model in PyTorch.
    
<br>

- Instructions:
    - Data Preparation: Starter code will be provided that loads the AG_NEWS dataset and prepares it for training. Do not modify this part. However, you should be sure to understand it, and comment it, the use of markdown cells is suggested. 

    - Model Setup: A skeleton code for the RNN model class will be provided. Complete this class and use it to instantiate your model.

    - Implementing Accuracy Function: Write a function that takes model predictions and ground truth labels as input and returns the model's accuracy.

    - Training Function: Implement a function that performs training on the given model using the AG_NEWS dataset. Your model should achieve an accuracy of at least 80% to get full marks for this part.

    - Text Sampling: Write a function that takes a sample text as input and classifies it using your trained model.

    - Confusion Matrix: Implement a function to display the confusion matrix for your model on the test data.

    - Submission: Submit your completed Jupyter Notebook. Make sure to include a markdown cell at the beginning of the notebook that lists the names of all team members. Teams should consist of 3 to 4 members.
    
<br>

- Evaluation Criteria:

    - Correct setup of all the required libraries and modules (10%)
    - Code Quality (30%): Your code should be well-organized, clearly commented, and easy to follow. Use also markdown cells for clarity. Comments should be given for all the provided code, this will help you understand its functionality.
    
   - Functionality (60%): 
        - All the functions should execute without errors and provide the expected outputs.
        - RNN model class (20%)
        - Accuracy fucntion (10%)
        - Training function (10%)
        - Sampling function (10%)
        - Confucion matrix (10%)

        - The model should achieve at least an 80% accuracy on the AG_NEWS test set for full marks in this criterion.


Dataset

https://pytorch.org/text/stable/datasets.html#text-classification

https://paperswithcode.com/dataset/ag-news


### Import libraries

In [ ]:
# conda install -c pytorch torchtext
# conda install -c pytorch torchdata
# conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch

In [1]:
# The following libraries are required for running the given code
# Please feel free to add any libraries you consider adecuate to complete the assingment.
import numpy as np
#PyTorch libraries
import torch
from torchtext.datasets import AG_NEWS
# Dataloader library
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
# neural layers
from torch import nn
from torch.nn import functional as F

# These libraries are suggested to plot confusion matrix
# you may use others
import scikitplot as skplt
import gc

In [2]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### Get the train and the test datasets and dataloaders

Classes:

* 1 - World

* 2 - Sports

* 3 - Business

* 4 - Sci/Tech

We will convert them to:

* 0 - World

* 1 - Sports

* 2 - Business

* 3 - Sci/Tech

In [3]:
train_dataset, test_dataset = AG_NEWS()
train_dataset, test_dataset = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)

In [35]:
train_dataset[0]

(1,
 '5 convicted of rapes on island founded by Bounty mutineers The mayor of the Pacific Ocean island of Pitcairn and his son were among five men found guilty Monday of a series of rapes and sexual assaults against girls on the island stretching back over four decades.')

In [4]:
# Get the tokeniser
# tokeniser object
tokeniser = get_tokenizer('basic_english')

def yield_tokens(data):
    for _, text in data:
        yield tokeniser(text)

In [5]:
# Build the vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=["<unk>"])
#set unknown token at position 0
vocab.set_default_index(vocab["<unk>"])

In [6]:
#test tokens
tokens = tokeniser('Welcome to TE3007')
print(tokens, vocab(tokens))

['welcome', 'to', 'te3007'] [3314, 4, 0]


In [7]:
NUM_TRAIN = int(len(train_dataset)*0.9)
NUM_VAL = len(train_dataset) - NUM_TRAIN

In [8]:
train_dataset, val_dataset = random_split(train_dataset, [NUM_TRAIN, NUM_VAL])

In [9]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

108000 12000 7600


In [11]:
# function passed to the DataLoader to process a batch of data as indicated
def collate_batch(batch):
    # Get label and text
    y, x = list(zip(*batch))
    
    # Create list with indices from tokeniser
    x = [vocab(tokeniser(text)) for text in x]
    x = [t + ([0]*(max_tokens - len(t))) if len(t) < max_tokens else t[:max_tokens] for t in x]

    # Prepare the labels, by subtracting 1 to get them in the range 0-3
    return torch.tensor(x, dtype=torch.int32), torch.tensor(y, dtype=torch.int32) - 1

In [12]:
labels =  ["World", "Sports", "Business", "Sci/Tech"]
max_tokens = 50
BATCH_SIZE = 256

In [13]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch, shuffle = True)

### Let us build our RNN model

In [14]:
EMBEDDING_SIZE = 64 # complete
NEURONS = 8 # complete
LAYERS = 2 # complete
NUM_CLASSES = len(labels)

In [15]:
class RNN_Model_1(nn.Module):
    def __init__(self, embed_size, hidden, layers, num_classes):
        super().__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), 
                                            embedding_dim=embed_size)
        
        self.rnn = nn.RNN(input_size=embed_size, hidden_size=hidden, num_layers=layers, batch_first=True)
                        # You may use PyTorch nn.GRU(), nn.RNN(), or nn.LSTM()
        
        self.fc = nn.Linear(hidden, num_classes)
        
    def forward(self, x):
        # implement forward pass. This function will be called when executing the model
        # _, hidden = self.rnn(x)
        # out = hidden[-1, :, :]
        # out = self.fc(out)
        embeddings = self.embedding_layer(x)
        output, hidden = self.rnn(embeddings, torch.randn(LAYERS, len(x), NEURONS))
        out = self.fc(output[:,-1])
        return out

In [31]:
def accuracy(model, loader):
    correct_count = 0
    total_count = 0
    cost = 0
    model.eval()
    model = model.to(device=device)
    with torch.no_grad():
        for i, (xi, yi) in enumerate(loader):
            xi = xi.to(device=device, dtype=torch.long)
            yi = yi.to(device=device, dtype=torch.long)
            scores = model(xi)
            cost += (F.cross_entropy(scores, yi)).item()
            _, pred = scores.max(dim=1)
            print(pred, yi)
            correct_count += (pred == yi).sum()
            total_count += pred.size(0)
        return cost/i, float(correct_count)/total_count

In [17]:
def train(model, optimiser, epochs=100):
    model = model.to(device=device)
    train_cost = 0.
    val_cost = 0.
    for epoch in range(epochs):
        train_correct_num  = 0.
        train_total = 0.
        train_cost_acum = 0
        for mb, (xi, yi) in enumerate(train_loader):
            model.train()
            xi = xi.to(device=device, dtype=torch.long)
            yi = yi.to(device=device, dtype=torch.long)
            scores = model(xi)
            cost = F.cross_entropy(input= scores, target=yi)
            optimiser.zero_grad()
            cost.backward()
            optimiser.step()
            
            train_correct_num += (torch.argmax(scores, dim=1) == yi.squeeze()).sum()
            train_total += scores.size(0)  
            
            train_cost_acum += cost.item()
        
        val_cost, val_acc = accuracy(model, val_loader)
        train_acc = float(train_correct_num)/train_total
        train_cost = train_cost_acum/mb
        # if epoch%20 == 0:            
        print(f'Epoch:{epoch}, train cost: {train_cost:.6f}, val cost: {val_cost:.6f},'
                      f' train acc: {train_acc:.4f}, val acc: {val_acc:4f},'
                      f' lr: {optimiser.param_groups[0]["lr"]:.6f}')

In [24]:
epochs = 20# define
lr = 0.001 # to do
# instantiate model
rnn_model = RNN_Model_1(EMBEDDING_SIZE, NEURONS, LAYERS, NUM_CLASSES)
optimiser = torch.optim.Adam(rnn_model.parameters(), lr=lr)


In [25]:
train(rnn_model, optimiser, epochs)

Epoch:0, train cost: 1.381724, val cost: 1.303376, train acc: 0.2822, val acc: 0.405417, lr: 0.001000
Epoch:1, train cost: 1.185985, val cost: 1.132263, train acc: 0.4369, val acc: 0.457500, lr: 0.001000
Epoch:2, train cost: 1.039517, val cost: 1.038653, train acc: 0.4887, val acc: 0.497583, lr: 0.001000
Epoch:3, train cost: 0.950730, val cost: 0.979485, train acc: 0.5308, val acc: 0.528500, lr: 0.001000
Epoch:4, train cost: 0.871500, val cost: 0.908425, train acc: 0.5801, val acc: 0.573750, lr: 0.001000
Epoch:5, train cost: 0.784117, val cost: 0.860238, train acc: 0.6299, val acc: 0.602167, lr: 0.001000
Epoch:6, train cost: 0.717716, val cost: 0.834460, train acc: 0.6580, val acc: 0.614167, lr: 0.001000
Epoch:7, train cost: 0.671743, val cost: 0.824031, train acc: 0.6792, val acc: 0.625000, lr: 0.001000
Epoch:8, train cost: 0.634574, val cost: 0.820629, train acc: 0.6981, val acc: 0.647000, lr: 0.001000
Epoch:9, train cost: 0.609241, val cost: 0.804212, train acc: 0.7112, val acc: 0.6

In [32]:
print(accuracy(rnn_model, test_loader))

tensor([3, 1, 3, 2, 1, 1, 3, 3, 0, 3, 3, 0, 1, 3, 3, 0, 3, 3, 2, 0, 3, 1, 0, 3,
        3, 1, 1, 1, 0, 2, 1, 0, 1, 3, 3, 1, 0, 2, 1, 1, 1, 3, 0, 2, 0, 3, 0, 1,
        2, 1, 1, 2, 1, 3, 1, 2, 3, 1, 1, 1, 1, 0, 0, 0, 2, 1, 3, 1, 2, 3, 0, 3,
        0, 1, 1, 3, 2, 2, 2, 3, 0, 3, 3, 0, 0, 1, 0, 3, 3, 1, 3, 1, 3, 0, 1, 1,
        3, 0, 3, 1, 0, 3, 3, 0, 1, 1, 0, 0, 0, 3, 3, 1, 0, 3, 0, 2, 0, 3, 3, 1,
        1, 3, 0, 2, 1, 3, 3, 0, 1, 3, 0, 2, 2, 0, 3, 2, 3, 1, 3, 0, 0, 2, 3, 0,
        0, 1, 0, 0, 3, 3, 3, 2, 3, 0, 3, 1, 0, 3, 2, 1, 3, 1, 3, 1, 1, 2, 3, 0,
        2, 0, 3, 1, 1, 2, 2, 1, 3, 3, 3, 2, 3, 1, 3, 3, 1, 3, 0, 1, 2, 0, 0, 0,
        2, 3, 0, 0, 0, 0, 0, 3, 2, 1, 2, 1, 3, 3, 1, 2, 0, 1, 3, 1, 3, 3, 2, 1,
        3, 2, 3, 3, 0, 0, 0, 3, 0, 2, 1, 3, 2, 0, 0, 0, 0, 0, 3, 1, 2, 1, 3, 1,
        3, 1, 2, 1, 2, 0, 3, 0, 1, 3, 2, 3, 1, 0, 3, 2]) tensor([3, 1, 3, 0, 1, 1, 3, 3, 0, 3, 3, 0, 3, 3, 1, 0, 2, 3, 2, 0, 3, 1, 0, 3,
        3, 1, 3, 1, 0, 0, 1, 0, 1, 2, 3, 1, 0, 0, 1, 1, 1, 2, 2

In [ ]:
def sample_text(model, loader):
    pass

In [ ]:
sample_text(rnn_model, test_loader)

In [ ]:
# create confusion matrix
pass